In [5]:
# STEP 1: Import Libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import joblib

In [6]:
# STEP 2: Set Parameters
STOCK = "GOOG"               # stock symbol
START = "2012-01-01"         # start date
END = "2022-12-31"           # end date
TIME_STEPS = 100             # number of past days to use for prediction

In [7]:
# STEP 3: Download Stock Data
data = yf.download(STOCK, START, END)
data = data[['Close']]  # only closing prices

C:\Users\ASUS\AppData\Local\Temp\ipykernel_348\2844897323.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(STOCK, START, END)
[*********************100%***********************]  1 of 1 completed


In [8]:
# STEP 4: Train-Test Split (80:20)
train_size = int(len(data) * 0.80)
data_train = data[:train_size]
data_test = data[train_size:]

In [9]:
# STEP 5: Scale Data
scaler = MinMaxScaler(feature_range=(0, 1))
data_train_scaled = scaler.fit_transform(data_train)

In [10]:
# STEP 6: Create Training Sequences
x_train, y_train = [], []
for i in range(TIME_STEPS, data_train_scaled.shape[0]):
    x_train.append(data_train_scaled[i - TIME_STEPS:i])
    y_train.append(data_train_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [11]:
# STEP 7: Build LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(TIME_STEPS, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# STEP 8: Compile & Train Model
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)


Epoch 1/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - loss: 0.0117
Epoch 2/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 0.0024
Epoch 3/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 0.0020
Epoch 4/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - loss: 0.0019
Epoch 5/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0018
Epoch 6/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0020
Epoch 7/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.0018
Epoch 8/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.0018
Epoch 9/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0017
Epoch 10/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0015
Epoch 11/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 0.0014
Epoch 12/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.0015
Epoch 13/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0013
Epoch 14/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - loss: 0.0017
Epoch 15/50
67/67 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 0.0012
Epoc

In [13]:
# STEP 9: Save Model and Scaler
model.save("stock_prediction_model.keras")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [14]:
print("✅ Model and scaler saved: stock_prediction_model.keras, scaler.pkl")


✅ Model and scaler saved: stock_prediction_model.keras, scaler.pkl
